### Установка зависимостей

In [ ]:
# Базовое обновление инструментов
%pip install -U pip setuptools wheel

# PyTorch 
%pip install -U torch torchvision torchaudio

# Установщик OpenMMLab
%pip install -U openmim

# MMEngine и MMCV 2.1.x
!mim install "mmengine>=0.10.0"
!mim install "mmcv==2.1.0"

# MMDetection для авто-детекции людей 3.2.x
!mim install "mmdet==3.2.0"

# MMPose 1.x
!mim install "mmpose==1.3.2"

In [ ]:
%pip install -U gdown imageio-ffmpeg --quiet
%pip install -q -U imageio imageio-ffmpeg

%pip install -U "numpy==2.2.6" cython

# снесём и пересоберём оба COCO-пакета
%pip uninstall -y xtcocotools pycocotools
%pip install --no-binary=xtcocotools,pycocotools --no-cache-dir xtcocotools pycocotools

# на всякий случай переустановим mmpose (поверх тех же версий)
%pip install -U --no-cache-dir "mmpose==1.3.2"

### Перезапускаем ядро, проверяем версии

In [ ]:
import importlib
import platform
import sys

import numpy as np
import torch
import xtcocotools._mask as _mask
from mmpose.apis import MMPoseInferencer


def _ver(pkg):
    try:
        m = importlib.import_module(pkg)
        return getattr(m, "__version__", "N/A")
    except Exception as e:
        return f"not installed ({e})"


print("Platform:", platform.platform())
print("Python:", sys.version)
print("\ntorch:", _ver("torch"))
print("torchvision:", _ver("torchvision"))
print("mmengine:", _ver("mmengine"))
print("mmcv:", _ver("mmcv"))
print("mmdet:", _ver("mmdet"))
print("\nMMPoseInferencer import OK")
print("mmpose:", _ver("mmpose"))
print("\nxtcocotools with numpy version:", _ver("numpy"))


print(
    "\nDevice selected:", "mps" if torch.backends.mps.is_available() else "cpu"
)

### Каталоги проекта

In [ ]:
from pathlib import Path

BASE_DIR = Path(".").resolve()
INPUT_DIR = BASE_DIR / "input"
OUTPUT_DIR = BASE_DIR / "output"
WEIGHTS_DIR = BASE_DIR / "weights"
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_DIR =", BASE_DIR)
print("INPUT_DIR =", INPUT_DIR)
print("OUTPUT_DIR =", OUTPUT_DIR)
print("WEIGHTS_DIR =", WEIGHTS_DIR)

# Попытка на фото

### 1. Скачиваем изображение

In [ ]:
import gdown
from pathlib import Path
from IPython.display import display
from PIL import Image

GD_URL = "https://drive.google.com/file/d/1RenekFPFYrB1UhAHhdZf67LeCUw8WbZh/view?usp=sharing"
img_path = INPUT_DIR / "cheliki_na_turnike.jpg"

gdown.download(GD_URL, str(img_path), quiet=False, fuzzy=True)
assert img_path.exists(), f"Изображение не скачалось: {img_path}"

print("Скачано в:", img_path)
print("Размер: {:.2f} МБ".format(img_path.stat().st_size / (1024**2)))

display(Image.open(img_path))

### 2. Применяем MMPoseInferencer на скачанном изображении

In [ ]:
from pathlib import Path
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
from mmpose.apis import MMPoseInferencer

device = "cpu"

try:
    torch.serialization.add_safe_globals(
        [
            np.core.multiarray._reconstruct,
            np.dtype,
            np.ufunc,
        ]
    )
except Exception:
    pass

print("Используем изображение:", img_path)

# Инициализируем унифицированный инференсер MMPose
# с alias "human" (детектор + оценка позы)
inferencer = MMPoseInferencer("human", device=device)

result_gen = inferencer(
    str(img_path),
    return_vis=True,  # вернуть массив визуализации в result['visualization']
    show=False,  # не открывать отдельное окно
    radius=16,
    thickness=8,  # сделать точки/скелет чуть заметнее
    vis_out_dir=str(
        OUTPUT_DIR / "vis"
    ),  # сюда сохранятся картинки с разметкой
    pred_out_dir=str(OUTPUT_DIR / "pred"),  # сюда — JSON с ключевыми точками
)

result = next(result_gen)

vis_list = result.get("visualization", [])
if not vis_list:
    raise RuntimeError(
        "Нет визуализации в результате. Убедитесь, что return_vis=True."
    )

vis_img = vis_list[
    0
]  # это RGB-изображение с нарисованными ключевыми точками/скелетом

# 5) Сохраним и покажем
out_file = OUTPUT_DIR / f"{img_path.stem}_pose_vis.jpg"
# cv2.imwrite ожидает BGR -> конвертируем
cv2.imwrite(str(out_file), vis_img[..., ::-1])

print("Визуализация сохранена в:", out_file)

plt.figure(figsize=(10, 8))
plt.imshow(vis_img)  # vis_img уже в RGB
plt.axis("off")
plt.title("MMPose: скелетики на изображении")
plt.show()

# Попытка на видео

### 1. Скачиваем видео и анализируем

In [ ]:
import os, cv2, gdown
from pathlib import Path


GD_URL = "https://drive.google.com/file/d/1iDHBbYgV_sYRyVUi_BLdqcGogD_CDXmB/view?usp=sharing"
video_pth = INPUT_DIR / "tiktonik_360p.mp4"

video_pth.unlink(missing_ok=True)
gdown.download(GD_URL, str(video_pth), quiet=False, fuzzy=True)
assert video_pth.exists(), f"Видео не скачалось: {video_pth}"


cap = cv2.VideoCapture(str(video_pth))
if not cap.isOpened():
    raise RuntimeError(f"Не удалось открыть видео: {video_pth}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
cap.release()

print(f"Видео: {video_pth}")
print(f"Размер: {w}x{h}, FPS: {fps:.3f}, кадров: {n}")

### 2. Применяем MMPoseInferencer на скачанном видео

In [ ]:
import cv2, imageio, numpy as np, torch, os
from pathlib import Path
from mmpose.apis import MMPoseInferencer

# Пути и устройство
device = "cpu"  # на macOS так избегаем проблемы NMS на MPS
out_dir = OUTPUT_DIR / "tiktonik_pose"

# Инициализируем единый инференсер с alias "human"
# (под капотом подтянет RTMDet для людей + 2D-позу; умеет видео/изображения;
# настраиваемые radius/thickness)
inferencer = MMPoseInferencer(
    "human",
    device=device,
)

# Готовим writer с кодеком H.264
vis_dir = out_dir / "visualization"
vis_dir.mkdir(parents=True, exist_ok=True)
out_video = vis_dir / (video_pth.stem + "_pose.mp4")

writer = imageio.get_writer(
    out_video.as_posix(),
    fps=fps,
    codec="libx264",
    format="FFMPEG",
    output_params=["-pix_fmt", "yuv420p"],
)

# Запускаем ленивый генератор инференса по видео.
# Чтобы получать кадры с отрисовкой в Python, включаем return_vis=True
# и задаем толщину/радиус. При желании можно включить/выключить рамки:
# draw_bbox=True/False.
result_gen = inferencer(
    str(video_pth),
    show=False,
    return_vis=True,
    radius=16,
    thickness=8,
    draw_bbox=False,
)

# Пробегаем все результаты и пишем в MP4
frames_written = 0
for res in result_gen:
    # 1) Заберём визуализацию (может быть списком или None)
    vis = res.get("visualization")
    if isinstance(vis, list):
        vis = vis[0] if vis else None

    # 2) Если отрисовка вернулась путём (бывает), читаем с диска
    if vis is None:
        vis_path = res.get("visualization_path")
        if isinstance(vis_path, list):
            vis_path = vis_path[0] if vis_path else None
        if vis_path:
            vis = imageio.v2.imread(vis_path)  # RGB

    if vis is None:
        continue  # нечего писать

    # 3) Приводим к RGB uint8 HxWx3
    frame = np.asarray(vis)
    if frame.ndim == 2:  # если ч/б
        frame = np.repeat(frame[..., None], 3, axis=2)
    elif (
        frame.ndim == 3 and frame.shape[2] == 4
    ):  # если RGBA — отбрасываем альфу
        frame = frame[..., :3]
    if frame.dtype != np.uint8:
        frame = np.clip(frame, 0, 255).astype(np.uint8)

    # 4) Подгоним размер под исходное видео (на всякий)
    if frame.shape[1] != w or frame.shape[0] != h:
        frame = cv2.resize(frame, (w, h), interpolation=cv2.INTER_LINEAR)

    # 5) Пишем в MP4 (imageio ожидает RGB)
    writer.append_data(frame)
    frames_written += 1
    if frames_written % 50 == 0:
        print(f"Обработано кадров: {frames_written}")

writer.close()
print(f"Готово! Сохранено кадров: {frames_written}")
print("Выходной файл:", out_video)